In [51]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn.metrics.pairwise import cosine_similarity
from scipy.optimize import minimize, rosen, rosen_der
from sklearn.model_selection import train_test_split

In [59]:
data = pd.read_csv("corpus/train.csv")
np_data = np.array(data)[:,3:]
data

,_unit_id,_trusted_judgments,comment,antagonise,antagonise:confidence,condescending,condescending:confidence,dismissive,dismissive:confidence,generalisation,generalisation:confidence,generalisation_unfair,generalisation_unfair:confidence,healthy,healthy:confidence,hostile,hostile:confidence,sarcastic,sarcastic:confidence
0,1739460326,5,proving there is no cure for stupidity.,1,0.5816,1,0.5816,1,0.5816,0,1.0000,0.0,1.0,0,0.5816,1,0.5816,0,0.8001
1,2297540155,5,Personally I prefer the Flying Spaghetti Monst...,0,1.0000,0,1.0000,0,1.0000,0,1.0000,0.0,1.0,1,0.7981,0,1.0000,0,1.0000
2,1812168131,5,Your comparing a pipeline to a well? One that ...,0,1.0000,0,0.8063,0,1.0000,0,1.0000,0.0,1.0,1,0.6081,0,1.0000,0,0.8063
3,1739470334,5,who is writing this pap!?,0,0.7931,0,0.5959,0,0.5959,0,1.0000,0.0,1.0,0,0.7917,0,1.0000,0,0.6052
4,1739466190,3,Natives refuse to even consider that their cur...,0,1.0000,0,1.0000,0,1.0000,0,1.0000,0.0,1.0,1,1.0000,0,1.0000,0,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35498,2028122744,5,Step 1 in building a fascist regime: get rid o...,0,0.8114,0,0.7888,0,0.8114,0,0.6002,0.0,0.0,1,0.6125,0,0.6002,0,0.6002
35499,1739457896,3,it takes months for political hate ads to have...,0,1.0000,0,1.0000,0,1.0000,0,1.0000,0.0,1.0,1,1.0000,0,1.0000,0,1.0000
35500,1812167305,5,As long as your hopes for converting the self-...,0,1.0000,0,1.0000,0,1.0000,0,1.0000,0.0,1.0,1,0.7978,0,1.0000,0,1.0000
35501,1739471203,3,but the comments section of wentes columns are...,0,1.0000,0,1.0000,0,1.0000,0,1.0000,0.0,1.0,1,1.0000,0,1.0000,0,1.0000


In [61]:
target_column = "healthy"
data = data.drop(columns = ["comment", "_unit_id"])

xtrain, xtest, ytrain, ytest = train_test_split(data.drop(columns=target_column), data[target_column])

In [39]:
np_data = np.array(all_data)
pos_sum = sum(all_data["healthy"])
print(f"""The total of comments is {len(all_data)}, of those {pos_sum} healthy, 
so saying everything is healthy would already result in a correct percentage of {pos_sum/ len(all_data)*100}%""")


The total of comments is 35503, of those 32848 healthy, 
so saying everything is healthy would already result in a correct percentage of 92.52175872461483%


In [66]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier(n_estimators=100)
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("rf", rnd_clf), ("svc", svm_clf)],
    voting="hard",
)
voting_clf.fit(xtrain, ytrain)

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(xtrain, ytrain)
    y_pred = clf.predict(xtest)
    print(clf.__class__.__name__, accuracy_score(ytest, y_pred))

/home/tim/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/tim/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

LogisticRegression 0.9400630914826499
RandomForestClassifier 0.9523433979269942
SVC 0.9427670121676431


/home/tim/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier 0.9453582694907616
